In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [103]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the {context}, identify the entities and relations, and write the corresponding Cypher query to create the knowledge graph. Use the following format for the query:

`CREATE (node1:Label1 {{property1: "value1", property2: "value2", ...}})-[relationship:RELATIONSHIP_TYPE {{propertyA: "valueA", propertyB: "valueB", ...}}]->(node2:Label2 {{property3: "value3", property4: "value4", ...}})`

Convert the context into nodes and relationships using the Cypher language. The output should only contain the complete queries.

Neo4j:
"""
prompt = ChatPromptTemplate.from_template(template)


In [104]:
from langchain_openai import OpenAI
from langchain_core.output_parsers import StrOutputParser

model = OpenAI()
output_parser = StrOutputParser()

In [118]:
file_path = 'files/love-story.txt'

with open(file_path, 'r') as file:
    content = file.read()

In [106]:
chain = prompt | model | output_parser
neo4j_syntax = chain.invoke({"context": content})

In [107]:
neo4j_syntax

'\nCREATE (Harry:Person {name: "Harry", occupation: "bookstore employee", personality: "shy, unassuming"})\nCREATE (Sophia:Person {name: "Sophia", occupation: "college student", personality: "curious, book lover"})\nCREATE (Bookstore:Place {name: "bookstore", location: "small college town"})\nCREATE (Autumn:Season {name: "autumn"})\nCREATE (Rain:Weather {name: "rain"})\nCREATE (Coffee:Drink {name: "coffee"})\nCREATE (Park:Place {name: "park"})\nCREATE (Creek:Place {name: "creek"})\nCREATE (Night:Time {name: "night"})\nCREATE (Stars:Sky {name: "stars"})\nCREATE (City:Place {name: "city"})\nCREATE (Wedding:Event {name: "wedding"})\nCREATE (Love:Feeling {name: "love"})\nCREATE (Confidence:Trait {name: "confidence"})\nCREATE (Purpose:Goal {name: "purpose"})\n\nCREATE (Harry)-[:WORKS_AT]->(Bookstore)\nCREATE (Sophia)-[:STUDENT_AT]->(Bookstore)\nCREATE (Sophia)-[:LOVES]->(Books)\n'

In [108]:
queries = [line.strip() for line in neo4j_syntax.split('\n') if line.strip()]
queries

['CREATE (Harry:Person {name: "Harry", occupation: "bookstore employee", personality: "shy, unassuming"})',
 'CREATE (Sophia:Person {name: "Sophia", occupation: "college student", personality: "curious, book lover"})',
 'CREATE (Bookstore:Place {name: "bookstore", location: "small college town"})',
 'CREATE (Autumn:Season {name: "autumn"})',
 'CREATE (Rain:Weather {name: "rain"})',
 'CREATE (Coffee:Drink {name: "coffee"})',
 'CREATE (Park:Place {name: "park"})',
 'CREATE (Creek:Place {name: "creek"})',
 'CREATE (Night:Time {name: "night"})',
 'CREATE (Stars:Sky {name: "stars"})',
 'CREATE (City:Place {name: "city"})',
 'CREATE (Wedding:Event {name: "wedding"})',
 'CREATE (Love:Feeling {name: "love"})',
 'CREATE (Confidence:Trait {name: "confidence"})',
 'CREATE (Purpose:Goal {name: "purpose"})',
 'CREATE (Harry)-[:WORKS_AT]->(Bookstore)',
 'CREATE (Sophia)-[:STUDENT_AT]->(Bookstore)',
 'CREATE (Sophia)-[:LOVES]->(Books)']

In [110]:
import os 
from neo4j import GraphDatabase

# Connect to the Neo4j database
URL = os.getenv("URL")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")

driver = GraphDatabase.driver(URL, auth=(USERNAME, PASSWORD))

# Function to execute a write query
def write_query(tx, query):
    tx.run(query)

# Execute the queries
with driver.session() as session:
    for query in queries:
        try:
            session.execute_write(write_query, query)
        except:
            print(f"Error with query: {query}")
# Close the connection
driver.close()

In [113]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI

graph = Neo4jGraph(url=URL, username=USERNAME, password=PASSWORD)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

chain.run("What does Sophia love?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Sophia"})-[:LOVES]->(d:Drink)
RETURN d.name
Full Context:
[]

> Finished chain.


"I don't know the answer."

In [115]:
chain.run("Who is Sophia?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Sophia"}) RETURN p;
Full Context:
[{'p': {'occupation': 'college student', 'personality': 'curious, book lover', 'name': 'Sophia'}}]

> Finished chain.


'Sophia is a college student who is curious and loves books.'